# Automated Machine Learning
**AutoML Forecasting Recipe Univariate**

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](../../../resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](../../../resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML time-series forecasting Job` with the 'forecasting()' factory-fuction
- Train the model using AmlCompute by submitting/running the AutoML forecasting training job
- Obtain the model and use it to generate forecast

**Motivations** - This notebook explains how to setup and run an AutoML forecasting job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'regression', 'classification', 'image classification', 'image object detection', 'nlp text classification', etc.

In this example we use the bike-sharing service dataset to showcase how you can use AutoML for a simple forecasting problem and explore the results.

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1. Import the required libraries

In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl
from azure.ai.ml import Input

import json
import pandas as pd
import os
import shutil
import yaml

### 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

#### Show Azure ML Workspace information

In [ ]:
credential = DefaultAzureCredential()
ml_client = None
# import pdb;pdb.set_trace()
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

In [ ]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
output

## 2. Data

We will load the data into DataFrame objects, split the data to train and test datasets, then create the Azure Machine Learning MLTable objects to prepare for the later training and inference steps.

### 2.1 Load the data file into DataFrame

We will load the data on local machine in dataframe objects. Only for this experiment, the data is in data folder, uploaded manually. We will drop the Covid period.

In [5]:
TARGET_COLNAME = "S4248SM144SCEN"
TIME_COLNAME = "observation_date"
COVID_PERIOD_START = (
    "2020-03-01"  # start of the covid period. To be excluded from evaluation.
)

df = pd.read_csv("./data/S4248SM144SCEN.csv", parse_dates=[TIME_COLNAME])
# df[TIME_COLNAME] = pd.to_datetime(df[TIME_COLNAME], format="%Y-%m-%d")
df.sort_values(by=TIME_COLNAME, inplace=True)

# remove the Covid period
df = df.query('{} <= "{}"'.format(TIME_COLNAME, COVID_PERIOD_START))

#### Set parameters

The first set of parameters is based on the analysis performed in the auto-ml-forecasting-univariate-recipe-experiment-settings notebook.

In [6]:
# set parameters based on the settings notebook analysis
DIFFERENCE_SERIES = True
TARGET_LAGS = None
STL_TYPE = None

In [7]:
# set other parameters
FORECAST_HORIZON = 12
TIME_SERIES_ID_COLNAMES = []
BLOCKED_MODELS = []

Next, define additional parameters to be used in the AutoML config class.

<li>FORECAST_HORIZON: The forecast horizon is the number of periods into the future that the model should predict. It is generally recommended that users set forecast horizons to less than 100 time periods. Furthermore, **AutoML's memory use and computation time increase in proportion to the length of the horizon**, so consider carefully how this value is set. If a long horizon forecast really is necessary, consider aggregating the series to a coarser time scale. Here, we set the horizon to 12 periods (i.e. 12 quarters). </li>
<li>TIME_SERIES_ID_COLNAMES: The names of columns used to group a timeseries. It can be used to create multiple series. If time series identifier is not defined, the data set is assumed to be one time-series. This parameter is used with task type forecasting. Since we are working with a single series, this list is empty.</li>

Learn more about forecast horizons in our [Auto-train a time-series forecast model](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-auto-train-forecast#configure-and-run-experiment) guide. For more discussion of forecast horizons and guiding principles for setting them, please see the energy demand notebook .

In [8]:
# difference data and test for unit root
if DIFFERENCE_SERIES:
    df_delta = df.copy()
    df_delta[TARGET_COLNAME] = df[TARGET_COLNAME].diff()
    df_delta.dropna(axis=0, inplace=True)

### Split the data into train and test

In [9]:
def ts_train_test_split(df_input, n, time_colname, ts_id_colnames=None):
    """
    Group data frame by time series ID and split on last n rows for each group.

    :param df_input: input data frame
    :param n: number of observations in the test set
    :param time_colname: time column
    :param ts_id_colnames: (optional) list of grain column names
    :return train and test data frames
    """
    if ts_id_colnames is None:
        ts_id_colnames = []
    ts_id_colnames_original = ts_id_colnames.copy()
    if len(ts_id_colnames) == 0:
        ts_id_colnames = ["Grain"]
        df_input[ts_id_colnames[0]] = "dummy"
    # Sort by ascending time
    df_grouped = df_input.sort_values(time_colname).groupby(
        ts_id_colnames, group_keys=False
    )
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    # drop group column name if it was not originally provided
    if len(ts_id_colnames_original) == 0:
        df_head.drop(ts_id_colnames, axis=1, inplace=True)
        df_tail.drop(ts_id_colnames, axis=1, inplace=True)
    return df_head, df_tail

In [10]:
# split the data into train and test set
if DIFFERENCE_SERIES:
    # generate train/inference sets using data in first differences
    df_train, df_test = ts_train_test_split(
        df_input=df_delta,
        n=FORECAST_HORIZON,
        time_colname=TIME_COLNAME,
        ts_id_colnames=TIME_SERIES_ID_COLNAMES,
    )
else:
    df_train, df_test = ts_train_test_split(
        df_input=df,
        n=FORECAST_HORIZON,
        time_colname=TIME_COLNAME,
        ts_id_colnames=TIME_SERIES_ID_COLNAMES,
    )

In [11]:
# Save the DataFrame objects to files
train_data_path = "./data/train_S4248SM144SCEN.csv"
test_data_path = "./data/test_S4248SM144SCEN.csv"
df_train.to_csv(train_data_path, index=False)
df_test.to_csv(test_data_path, index=False)

### Create the Azure Machine Learning MLTable

With Azure Machine Learning MLTables you can keep a single copy of data in your storage, easily access data during model training, share data and collaborate with other users. 
Below, we will upload the data by creating an MLTable to be used for training.

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [12]:
def create_folder_and_ml_table(csv_file, output, delimiter=",", encoding="ascii"):
    os.makedirs(output, exist_ok=True)
    fname = os.path.split(csv_file)[-1]

    mltable = {
        "paths": [{"file": f"./{fname}"}],
        "transformations": [
            {"read_delimited": {"delimiter": delimiter, "encoding": encoding}}
        ],
    }
    with open(os.path.join(output, "MLTable"), "w") as f:
        f.write(yaml.dump(mltable))
    shutil.copy(csv_file, os.path.join(output, fname))

In [13]:
train_mltable_path = "./data/training-mltable-folder"
create_folder_and_ml_table(train_data_path, train_mltable_path)

In [14]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=train_mltable_path)

Next, we upload the directory with the test set data which will be used in the batch end point inference.

In [15]:
test_mltable_path = "./data/test-mltable-folder"
create_folder_and_ml_table(test_data_path, test_mltable_path)

In [16]:
my_test_data_input = Input(type=AssetTypes.MLTABLE, path=test_mltable_path)

## 3. Create or Attach existing AmlCompute

[Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) is a managed-compute infrastructure that allows the user to easily create a single or multi-node compute. In this tutorial, you will create and an AmlCompute cluster as your training compute resource.

<b>Creation of AmlCompute takes approximately 5 minutes.</b>

If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [17]:
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.ml.entities import AmlCompute

cluster_name = "recipe-cluster"

try:
    # Retrieve an already attached Azure Machine Learning Compute.
    compute = ml_client.compute.get(cluster_name)
except ResourceNotFoundError as e:
    compute = AmlCompute(
        name=cluster_name,
        size="STANDARD_DS12_V2",
        type="amlcompute",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=120,
    )
    poller = ml_client.begin_create_or_update(compute)
    # poller.wait()

## 4. Configure and run the AutoML Forecasting training job

In this section we will configure and run the AutoML job to train the model.

### 4.1 Configure the job through the forecasting() factory function

#### forecasting() function parameters:

The `forecasting()` factory function allows user to configure AutoML for the forecasting task for the most common scenarios with the following properties.

|Property|Description|
|-|-|
|**target_column_name**|The name of the label column.|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**training_data**|The training data to be used for this experiment. You can use a registered MLTable in the workspace using the format `<mltable_name>:<version>` OR you can use a local file or folder as a MLTable. For e.g `Input(mltable='my_mltable:1')` OR `Input(mltable=MLTable(local_path="./data"))` The parameter 'training_data' must always be provided.|
|**compute**|The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it with any other compute in the workspace.|
|**n_cross_validations**|Number of cross-validation folds to use for model/pipeline selection. The default value is "auto", in which case AutoMl determines the number of cross-validations automatically, if a validation set is not provided. Or, users could specify an integer value.|
|**name**|The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
|**experiment_name**|The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment. For example, if a user runs this notebook multiple times, there will be multiple runs associated with the same Experiment name.|
|**enable_model_explainability**|If set to true, the explanations such as feature importance for the best model will be generated.|

#### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.

|Property|Description|
|-|-|
|**timeout_minutes**|Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results. It is hard to say what the timeout limit should be because the runtimes depend on multiple factors such as number of unique time series in the dataset, length of time series, statistical properties of the data, etc. If your dataset is less than 10,000,000 observations, you can try to set the experiment to 1 hour. If you are seeing less than 30 child jobs completed in this time frame, increase the timeout limit and re-run the experiment.|
|**trial_timeout_minutes**|Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.|
|**max_trials**|The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If you are setting the `enable_early_termination=True` the number of trials will be smaller.|
|**max_concurrent_trials**|Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to set this number equal to the number of nodes in your cluster.|
|**enable_early_termination**|Whether to enable early termination if the score is not improving over 10 iterations. Early stopping window starts only after first 20 iterations. This means that the first iteration where stopping can occur is the 31st.|

#### Specialized Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the .set_forecast_settings() method. 
The table below details the forecasting parameters we will be passing into our experiment.

|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**frequency**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.
|**cv_step_size**|Number of periods between two consecutive cross-validation folds. The default value is "auto", in which case AutoML determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value.|
|**target_lags**|The target_lags specifies how far back we will construct the lags of the target variable.|
|**country_or_region_for_holidays**|The country/region used to generate holiday features. These should be ISO 3166 two-letter country/region codes (i.e. 'US', 'GB').|

#### Using lags and rolling window features
This training is also using the **target lags** which are lagged values of the target variable. Since we generate lags with respect to horizon, we therefore must still specify the `forecast_horizon` that the model will learn to forecast. The `target_lags` keyword specifies how far back we will construct the lags of the target variable. The `target_lags` is set to auto in this notebook. Therefore, this value will be computed automatically.

This notebook uses the .set_training(blocked_training_algorithms=...) parameter to exclude some models that take a longer time to train on this dataset.  You can choose to remove models from the blocked_training_algorithms list but you may need to increase the trial_timeout_minutes parameter value to get results.

To run AutoML, you need to create an Experiment. 

In [18]:
# choose a name for the run history container in the workspace
if isinstance(TARGET_LAGS, list):
    TARGET_LAGS_STR = (
        "-".join(map(str, TARGET_LAGS)) if (len(TARGET_LAGS) > 0) else None
    )
else:
    TARGET_LAGS_STR = TARGET_LAGS

experiment_desc = "diff-{}_lags-{}_STL-{}".format(
    DIFFERENCE_SERIES, TARGET_LAGS_STR, STL_TYPE
)

# General job parameters.
max_trials = 5
exp_name = "univariate_recipe_{}".format(experiment_desc)

### 4.2 Create the AutoML forecasting job

In [ ]:
# Create the AutoML forecasting job with the related factory-function.

forecasting_job = automl.forecasting(
    compute=cluster_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name=TARGET_COLNAME,
    primary_metric="NormalizedRootMeanSquaredError",
    n_cross_validations=3,
    enable_model_explainability=True,
    tags={"my_custom_tag": "My custom value"},
)

# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)
# Featurization properties are optional
# forecasting_job.set_featurization(# drop_columns=["not_needed_column"], # Optional
#                                   # enable_dnn_featurization=True         # Enable if there are text columns
#                                     )

In [20]:
# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
    time_column_name=TIME_COLNAME,
    forecast_horizon=FORECAST_HORIZON,
    target_lags=TARGET_LAGS,
    time_series_id_column_names=TIME_SERIES_ID_COLNAMES,
    use_stl=STL_TYPE,
    # ADDITIONAL FORECASTING TRAINING PARAMS ---
    # frequency="H",
    # time_series_id_column_names=["tid1", "tid2", "tid2"],
    # short_series_handling_config=ShortSeriesHandlingConfiguration.DROP,
    # use_stl="season",
    # seasonality=3,
    # target_rolling_window_size=4,
)

# Training properties are optional
forecasting_job.set_training(blocked_training_algorithms=["ExtremeRandomTrees"])

### 4.2 Train the AutoML model

Using the `MLClient` created earlier, we will execute the following commands to train the model.

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    forecasting_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
# Wait until AutoML training runs are finished
ml_client.jobs.stream(returned_job.name)

## 5. Retrieve the Best Trial (Best Model's trial/run)

Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

### 5.1 Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow


#### Obtain the tracking URI for MLFlow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

In [25]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

#### Get the AutoML parent Job

In [ ]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "591640e8-0f88-49c5-adaa-39b9b9d75531"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

In [ ]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

#### Get the AutoML best child run

In [ ]:
# Get the best model's child run
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

### 5.2 Get best model run's validation metrics

Access the results (such as models, artifacts, metrics) of a previously completed AutoML Run.



In [ ]:
pd.DataFrame(best_run.data.metrics, index=[0]).T

## 6. Model evaluation and deployemnt.

### 6.1 Download the best model
Access the results (such as models, artifacts, metrics) of a previously completed AutoML Run.

In [ ]:
# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

In [ ]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

#### Featurization
We can look at the engineered feature names generated in time-series featurization via the JSON file named 'engineered_feature_names.json' under the run outputs.

In [ ]:
import json

with open(os.path.join(local_path, "engineered_feature_names.json"), "r") as f:
    records = json.load(f)

print(records)

You can also see what featurization steps were performed on different raw features in the user data. For each raw feature in the user data, the following information is displayed:

+ Raw feature name
+ Number of engineered features formed out of this raw feature
+ Type detected
+ If feature was dropped
+ List of feature transformations for the raw feature

In [ ]:
# Render the JSON as a pandas DataFrame
with open(os.path.join(local_path, "featurization_summary.json"), "r") as f:
    records = json.load(f)
fs = pd.DataFrame.from_records(records)

# View a summary of the featurization
fs[
    [
        "RawFeatureName",
        "TypeDetected",
        "Dropped",
        "EngineeredFeatureCount",
        "Transformations",
    ]
]

### 6.2 Forecasting using batch endpoint

Now that we have retrieved the best pipeline/model, it can be used to make predictions on test data. We will do batch inferencing on the test dataset which must have the same schema as training dataset.

The inference will run on a remote compute. In this example, it will re-use the training compute.

#### 6.2.1 Create a model endpoint
First, we need to register the model, environment and the batch endpoint.

In [ ]:
from azure.ai.ml.entities import (
    Environment,
    BatchEndpoint,
    BatchDeployment,
    BatchRetrySettings,
    Model,
)
from azure.ai.ml.constants import BatchDeploymentOutputAction

model_name = "forecasting-recipe-univariate"
batch_endpoint_name = "forecasting-recipe-univariate"

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description="Forecasting Recipe Univariate Model",
)
registered_model = ml_client.models.create_or_update(model)

env = Environment(
    name="automl-tabular-env",
    description="environment for automl inference",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    conda_file="artifact_downloads/outputs/conda_env_v_1_0_0.yml",
)

endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="this is a sample batch endpoint",
)
ml_client.begin_create_or_update(endpoint)

To create a batch deployment, we will use the forecasting_script.py which will load the model and will call the forecast method each time we will envoke the endpoint.

In [35]:
output_file = "forecasting_recipe_univariate_output.json"
batch_deployment = BatchDeployment(
    name="univariate-non-mlflow-deployment",
    description="this is a sample non-mlflow deployment",
    endpoint_name=batch_endpoint_name,
    model=registered_model,
    code_path="./forecast",
    scoring_script="forecasting_script.py",
    environment=env,
    environment_variables={
        "TARGET_COLUMN_NAME": TARGET_COLNAME,
    },
    compute=cluster_name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name=output_file,
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
)

Finally, start a model deployment.

In [ ]:
ml_client.begin_create_or_update(batch_deployment)

We need to create the Input, representing URI folder, because the batch endpoint is intended to process multiple files at a time. In this example we will use only one test file, which we have uploaded to the blob storage earlier. This file must be available through the url link.

#### Create an inference job

In [37]:
os.makedirs("test_dataset", exist_ok=True)
shutil.copy(
    "data/test-mltable-folder/test_S4248SM144SCEN.csv",
    "test_dataset/test_S4248SM144SCEN.csv",
)

'test_dataset/test_S4248SM144SCEN.csv'

In [38]:
my_test_data_input = Input(
    type=AssetTypes.URI_FOLDER,
    path="test_dataset/",
)

In [39]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=batch_endpoint_name,
    input=my_test_data_input,
    deployment_name="univariate-non-mlflow-deployment",  # name is required as default deployment is not set
)

We will stream the job output to monitor the execution.

In [ ]:
job_name = job.name
batch_job = ml_client.jobs.get(name=job_name)
print(batch_job.status)
# stream the job logs
ml_client.jobs.stream(name=job_name)

#### Download the prediction result for metrics calculation

The output of forecast output is saved in JSON format. You can use it to calculate test set metrics and plot predictions and actuals over time.

In [ ]:
ml_client.jobs.download(job_name, download_path=".")

In [ ]:
pred_df = pd.read_json(output_file, orient="table")
pred_df.head()

In [43]:
def convert_fcst_diff_to_levels(fcst, yt, df_orig):
    """Convert forecast from first differences to levels."""
    fcst = fcst.reset_index(drop=False, inplace=False)
    fcst["predicted_level"] = fcst["predicted"].cumsum()
    fcst["predicted_level"] = fcst["predicted_level"].astype(float) + float(yt)
    # merge actuals
    out = pd.merge(
        fcst, df_orig[[TIME_COLNAME, TARGET_COLNAME]], on=[TIME_COLNAME], how="inner"
    )
    out.rename(columns={TARGET_COLNAME + "_y": "actual_level"}, inplace=True)
    return out

In [ ]:
if DIFFERENCE_SERIES:
    # convert forecast in differences to the levels
    INFORMATION_SET_DATE = max(df_train[TIME_COLNAME])
    YT = df.query("{} == @INFORMATION_SET_DATE".format(TIME_COLNAME))[TARGET_COLNAME]
    fcst_df = convert_fcst_diff_to_levels(fcst=pred_df, yt=YT, df_orig=df)
else:
    fcst_df = pred_df.copy()
    fcst_df["actual_level"] = y_test
    fcst_df["predicted_level"] = y_predictions

#### Calculate the metrics

In [ ]:
from metrics_helper import calculate_metrics

calculate_metrics(fcst_df["actual_level"], fcst_df["predicted_level"])

#### Forecast versus actuals plot
We will join actual observations of test data with the predictions to plot predictions and actuals on a time series plot.

In [46]:
plot_df = fcst_df[[TIME_COLNAME, "actual_level", "predicted_level"]]
plot_df.set_index(TIME_COLNAME, inplace=True)

###### Build the plot

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

lines = plot_df.plot.line()

In [ ]:
# Delete the batch endpoint and compute. Do not do it occasionally.
ml_client.batch_endpoints.begin_delete(name=batch_endpoint_name)
ml_client.compute.begin_delete(name=cluster_name)

### 6.3 Deployment
After we have tested our model on the batch endpoint, we may want to deploy it as a service. Currently no code deployment using mlflow is not supported for forecasting tasks and we will use the workaround which is described in the Deployment section of the [automl-forecasting-task-energy-demand notebook](https://github.com/Azure/azureml-examples/blob/main/sdk/python/jobs/automl-standalone-jobs/automl-forecasting-task-energy-demand/automl-forecasting-task-energy-demand-advanced-mlflow.ipynb).